In [2]:
import numpy as np # Successfully installed numpy-1.23.5
import pandas as pd # Successfully installed pandas-1.3.5
import matplotlib.pyplot as plt
import seaborn as sns
import os
import networkx as nx

import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Dataset
from torch_geometric.data import Data
from torch_geometric.nn import global_mean_pool
import sys
# Add the parent directory to the system path
# sys.path.append(os.path.abspath('..'))
# from models import GNN
# the temporal package is not available TODO: find what the issue is
# from torch_geometric_temporal.nn.recurrent import A3TGCN2 # torch-geometric-temporal 0.54.0 requires pandas<=1.3.5, but you have pandas 2.2.3 which is incompatible.
#from torch_geometric_temporal.signal import temporal_signal_split
# import 
# Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
# from stellargraph import StellarGraph
# import custom functions from utils.py
# from utils import *
# os.getcwd() # double check current working directory

In [4]:
# .. goes up one level in directory
path = r'../datasets/Car-Hacking Dataset/Fuzzy_dataset.csv'
path = r'../datasets/Car-Hacking Dataset/DoS_dataset.csv'
path = r'../datasets/Car-Hacking Dataset/gear_dataset.csv'
path = r'../datasets/Car-Hacking Dataset/RPM_dataset.csv'
df = pd.read_csv(path)
df.columns = ['Timestamp', 'CAN ID','DLC','Data1','Data2','Data3','Data4','Data5','Data6','Data7','Data8', 'label'] 

In [98]:
'''
Temporal graph dataset: a graph that changes over time
Graph type: Homogeneous graph
Nodes: CAN ID
Edges: The next CAN ID in the sequence
Node Features (optional): The data in the CAN ID

Steps:
1. identify node properties
2. Edges: How to connect nodes?
3. Extract labels

Convert to pytorch geometric format
One graph to another graph: 200 messages to 1 graph

There is also pytorch geometric temporal package

# build a baseline model RF, SVM, etc

# fraud datasets may have incorrect labels -> use label smoothing
Featureless nodes: use random numbers

make a GCN that uses labels only, and one that takes an average of data form its neighbors as well
Graph Attention Networks (GAT)

Idea: could I use FL on the different car datasets, share those weights and perhaps it could generalize better. A practical use case would be between different companies, where they don't want to share their data
but it is in their best interest to have the best performing cybersecurity model.

https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.explain.algorithm.GNNExplainer.html
'''

"\nTemporal graph dataset: a graph that changes over time\nGraph type: Homogeneous graph\nNodes: CAN ID\nEdges: The next CAN ID in the sequence\nNode Features (optional): The data in the CAN ID\n\nSteps:\n1. identify node properties\n2. Edges: How to connect nodes?\n3. Extract labels\n\nConvert to pytorch geometric format\nOne graph to another graph: 200 messages to 1 graph\n\nThere is also pytorch geometric temporal package\n\n# build a baseline model RF, SVM, etc\n\n# fraud datasets may have incorrect labels -> use label smoothing\nFeatureless nodes: use random numbers\n\nmake a GCN that uses labels only, and one that takes an average of data form its neighbors as well\nGraph Attention Networks (GAT)\n\nIdea: could I use FL on the different car datasets, share those weights and perhaps it could generalize better. A practical use case would be between different companies, where they don't want to share their data\nbut it is in their best interest to have the best performing cybersecur

In [5]:
df['Node'] = df['CAN ID']
df['Edge'] = df['CAN ID'].shift(-1)

In [6]:
def hex_to_decimal(x):
    if x is None or x == 'None':
        return None
    try:
        return int(x, 16)
    except (ValueError, TypeError):
        return x


In [7]:
# hex_to_decimal = lambda y: int(y, 16)
# Need to encode the strings to integers
df = df.apply(lambda x: x.apply(hex_to_decimal))

In [8]:
# Drop the last row
df = df.drop(df.index[-1])

In [103]:
# import cProfile
# import pstats
# import io

# # Profile the TimeSeriesGraphDataset creation
# pr = cProfile.Profile()
# pr.enable()

# # Create the TimeSeriesGraphDataset
# dataset = TimeSeriesGraphDataset(arr, window_size=50, stride=1)

# pr.disable()
# s = io.StringIO()
# sortby = 'cumulative'
# ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
# ps.print_stats()
# print(s.getvalue())

In [9]:
df

,Timestamp,CAN ID,DLC,Data1,Data2,Data3,Data4,Data5,Data6,Data7,Data8,label,Node,Edge
0,1.478191e+09,399,8,254,59,0,0.0,0.0,60.0,0.0,0.0,R,399,608.0
1,1.478191e+09,608,8,25,34,34,48.0,255.0,143.0,110.0,63.0,R,608,672.0
2,1.478191e+09,672,8,96,0,131,29.0,150.0,2.0,189.0,0.0,R,672,809.0
3,1.478191e+09,809,8,220,184,126,20.0,17.0,32.0,0.0,20.0,R,809,1349.0
4,1.478191e+09,1349,8,216,0,0,131.0,0.0,0.0,0.0,0.0,R,1349,704.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4621695,1.478201e+09,790,8,5,33,48,10.0,33.0,30.0,0.0,111.0,R,790,399.0
4621696,1.478201e+09,399,8,254,89,0,0.0,0.0,65.0,0.0,0.0,R,399,608.0
4621697,1.478201e+09,608,8,24,33,33,48.0,8.0,143.0,109.0,25.0,R,608,672.0
4621698,1.478201e+09,672,8,36,0,154,29.0,151.0,2.0,189.0,0.0,R,672,809.0


In [10]:
# Drop rows where 'Data3' contains 'R'
df_dropped = df[df['Data3'] != 'R']
df= df_dropped

In [106]:
# # Identify rows where 'Data3' contains 'R'
# mask = df['Data3'] == 'R'

# # Pad these rows with zeros and add 'R' label at the end
# df.loc[mask, :] = df.loc[mask, :].fillna(0)
# df.loc[mask, 'label'] = 'R'

In [11]:
df['label'] = df['label'].replace({'R': 0, 'T': 1})

C:\Users\rober\AppData\Local\Temp\ipykernel_5248\4091684753.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['label'] = df['label'].replace({'R': 0, 'T': 1})
C:\Users\rober\AppData\Local\Temp\ipykernel_5248\4091684753.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = df['label'].replace({'R': 0, 'T': 1})


In [12]:
# Replace NaN values with zero
df = df.fillna(0)

C:\Users\rober\AppData\Local\Temp\ipykernel_5248\63676179.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(0)


In [13]:
# Function to pad rows with zeros and add 'R' label at the end
def pad_row(row):
    if row['Data3'] == 'R':
        return pd.Series(np.append(row.fillna(0).values, 'R'))
    return row

# Apply the function to each row
# df_padded = df.apply(pad_row, axis=1)

# # Set pandas display option to show all rows
# # pd.set_option('display.max_rows', None)

# # Print the DataFrame
# print(df_padded)

In [14]:
# # Extract edge indices
edge_index = torch.tensor(df[['Node', 'Edge']].values, dtype=torch.long)

# # Extract node features (assuming features are in columns 'feature1', 'feature2', ..., 'featureN')
node_features = torch.tensor(df[['Data1','Data2','Data3','Data4','Data5','Data6','Data7','Data8',]].values, dtype=torch.float)

y = torch.tensor(df['label'], dtype=torch.long)
# # Create a Data object
data = Data(x=node_features, edge_index=edge_index, y=y)

In [15]:
data

Data(x=[4580224, 8], edge_index=[4580224, 2], y=[4580224])

In [ ]:
class TimeSeriesGraphDataset(Dataset):
    def __init__(self, time_series_data, window_size, stride, indices=None):
        self.data = time_series_data
        self.window_size = window_size
        self.stride = stride
        self.graphs = self._create_graphs()
        # torch_geometric.data.Dataset expects _indicies to be defined
        self._indices = indices if indices is not None else range(len(self.graphs))
        self.transform = None
        self.pre_transform = None

    def _create_graphs(self):
        graphs = []
        for i in range(0, len(self.data) - self.window_size + 1, self.stride):
            window = self.data[i:i+self.window_size]
            graph = self.create_single_graph(window)
            graphs.append(graph)
        return graphs

    def create_single_graph(self, window_data):
        x = torch.tensor(window_data[:, 0:1], dtype=torch.float)
        # call the edge index function here
        edge_index = self._get_edge_index(window_data)
        # last column are the labels
        y = torch.tensor([1 if 1 in window_data[:, -1] else 0], dtype=torch.long)
        return Data(x=x, edge_index=edge_index, y=y)
    
    def _get_edge_index(self, window_data: np.ndarray) -> torch.Tensor:
        num_nodes = window_data.shape[0]
        edge_index = torch.tensor([np.arange(num_nodes - 1), np.arange(1, num_nodes)], dtype=torch.long)
        return edge_index 

    def len(self):
        return len(self.graphs)

    def get(self, idx):
        return self.graphs[idx]

In [17]:
def create_graphs(data, window_size, stride):
    graphs = []
    for i in range(0, len(data) - window_size + 1, stride):
        window = data[i:i+window_size]
        graph = create_single_graph(window)
        graphs.append(graph)
    return graphs

def create_single_graph(window_data):
        x = torch.tensor(window_data[:, 0:1], dtype=torch.float)
        
        edge_index = _get_edge_index(window_data) # call the edge index function here
        
        label = window_data[:, -1] # last column are the labels
        y = torch.tensor([1 if np.any(label == 1) else 0], dtype=torch.long)
        return Data(x=x, edge_index=edge_index, y=y)

def _get_edge_index(window_data: np.ndarray) -> torch.Tensor:
        num_nodes = window_data.shape[0]
        edge_index = torch.tensor([np.arange(num_nodes - 1), np.arange(1, num_nodes)], dtype=torch.long)
        return edge_index

In [18]:
class GraphDataset(Dataset):
    def __init__(self, data_list):
        self.data_list = data_list

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        return self.data_list[idx]

In [19]:
arr = df[['Node', 'Edge', 'label']].to_numpy(dtype=float)

In [45]:
# Option 1: 2 Step process with functions and a simple GraphDataset class
# arr = df.to_numpy(dtype=float)
list_graphs = create_graphs(arr, window_size=50, stride=50)
# Create the dataset
dataset = GraphDataset(list_graphs)

In [46]:
dataset[0]

Data(x=[50, 1], edge_index=[2, 49], y=[1])

In [35]:
# Option 2: Use the TimeSeriesGraphDataset class
# Convert DataFrame to NumPy array with a specific data type

dataset2 = TimeSeriesGraphDataset(arr, window_size=50, stride=50)

In [47]:
dataset2[0]

Data(x=[50, 3], edge_index=[2, 49], y=[1])

In [22]:
dataset

GraphDataset(91604)

In [36]:
dataset2._indices
dataset2.transform
dataset2.pre_transform

In [156]:
list_graphs

[Data(x=[50, 1], edge_index=[2, 49], y=[1]),
 Data(x=[50, 1], edge_index=[2, 49], y=[1]),
 Data(x=[50, 1], edge_index=[2, 49], y=[1]),
 Data(x=[50, 1], edge_index=[2, 49], y=[1]),
 Data(x=[50, 1], edge_index=[2, 49], y=[1]),
 Data(x=[50, 1], edge_index=[2, 49], y=[1]),
 Data(x=[50, 1], edge_index=[2, 49], y=[1]),
 Data(x=[50, 1], edge_index=[2, 49], y=[1]),
 Data(x=[50, 1], edge_index=[2, 49], y=[1]),
 Data(x=[50, 1], edge_index=[2, 49], y=[1]),
 Data(x=[50, 1], edge_index=[2, 49], y=[1]),
 Data(x=[50, 1], edge_index=[2, 49], y=[1]),
 Data(x=[50, 1], edge_index=[2, 49], y=[1]),
 Data(x=[50, 1], edge_index=[2, 49], y=[1]),
 Data(x=[50, 1], edge_index=[2, 49], y=[1]),
 Data(x=[50, 1], edge_index=[2, 49], y=[1]),
 Data(x=[50, 1], edge_index=[2, 49], y=[1]),
 Data(x=[50, 1], edge_index=[2, 49], y=[1]),
 Data(x=[50, 1], edge_index=[2, 49], y=[1]),
 Data(x=[50, 1], edge_index=[2, 49], y=[1]),
 Data(x=[50, 1], edge_index=[2, 49], y=[1]),
 Data(x=[50, 1], edge_index=[2, 49], y=[1]),
 Data(x=[5

In [ ]:
'''' From GitHub Copilot
Key Attributes of the Data Class:
x: Node feature matrix with shape [num_nodes, num_node_features]. Each row corresponds to the feature vector of a node.
edge_index: Graph connectivity in COO format with shape [2, num_edges]. Each column represents an edge between two nodes.
edge_attr (optional): Edge feature matrix with shape [num_edges, num_edge_features]. Each row corresponds to the feature vector of an edge.
y (optional): Graph-level or node-level labels.
pos (optional): Node position matrix with shape [num_nodes, num_dimensions]. Useful for visualization or spatial graphs.
batch (optional): Batch vector, which assigns each node to a specific graph in a mini-batch.
'''
# label logic - if T in window then label is 1, else 0. Start simple and build out from there
# x = [# nodes, # features], for my case [200, 10]
# edge_index = [2, # edges], for my case [2, 199]
# y = [# nodes, # labels], for my case [200, 1]
# arr = df[['CAN ID', 'Node', 'Edge']].to_numpy()
# maybe a stride of 25 to speed up the process?? At least to start
# stride of 1 was over 10 GBs, so will increase to 25



C:\Users\User1\AppData\Local\Temp\ipykernel_9012\482898013.py:26: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\b\abs_e1nmgx0bk6\croot\pytorch-select_1725478824526\work\torch\csrc\utils\tensor_new.cpp:277.)
  edge_index = torch.tensor([np.arange(num_nodes - 1), np.arange(1, num_nodes)], dtype=torch.long)


In [17]:
path = r'../datasets/dataset_rpms_25.pt'
torch.save(dataset, path)

In [30]:
from collections import Counter

# Count the occurrences of each label
label_counts = Counter()
for data in dataset:
    label = data.y.item()
    label_counts[label] += 1

print("Label counts:", label_counts)

Label counts: Counter({0: 50369, 1: 41235})


In [ ]:
def train():
    model.train()
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data).squeeze()  # Squeeze the output to match the target shape
        loss = F.binary_cross_entropy_with_logits(out, data.y.float())
        loss.backward()
        optimizer.step()

def test(loader):
    model.eval()
    correct = 0
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            out = model(data).squeeze()  # Squeeze the output to match the target shape
            pred = (out > 0).long()
            correct += (pred == data.y).sum().item()
    return correct / len(loader.dataset)

In [44]:
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.loader import DataLoader
from torch.utils.data import random_split

class GNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        # GAT(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.fc = torch.nn.Linear(hidden_channels, out_channels)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch)  # Global mean pooling
        x = self.fc(x)
        return x

num_epochs = 2
dataset = dataset2
# dataset = dataset
# Define the split ratio
train_ratio = 0.8
test_ratio = 0.2

# Calculate the number of samples for training and testing
# Split the dataset into training and test sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# model = AnomalyGNN(num_node_features=50, hidden_channels=64)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
model = GNN(in_channels=1, hidden_channels=16, out_channels=1).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
criterion = nn.BCEWithLogitsLoss()

for epoch in range(num_epochs):
    for batch in train_loader:
        optimizer.zero_grad()
        out = model(batch).squeeze() 
        # print(out)
        # print(batch.y)
        loss = F.binary_cross_entropy_with_logits(out, batch.y.float())
        # loss = criterion(out, batch.y)
        loss.backward()
        optimizer.step()
    
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')
        

# RPM took 40 mins for 100 epochs. LR of 0.001 works much better than 0.01
# gear took 58 mins for 100 epochs. LR of 0.0005
# DoS took 40ish mins for 100 epochs. LR of 0.0005
# Fuzzy took 31ish mins for 100 epochs. LR of 0.0005

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1600x3 and 1x16)

In [ ]:
def train():
    model.train()
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data).squeeze()  # Squeeze the output to match the target shape
        loss = F.binary_cross_entropy_with_logits(out, data.y.float())
        loss.backward()
        optimizer.step()

def test(loader):
    model.eval()
    correct = 0
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            out = model(data).squeeze()  # Squeeze the output to match the target shape
            # print(out)
            pred = (out > 0).long()
            correct += (pred == data.y).sum().item()
    return correct / len(loader.dataset)

train_acc = test(train_loader)
test_acc = test(test_loader)
print(f'Train Accuracy: {train_acc:.4f}, Test Accuracy: {test_acc:.4f}')

Train Accuracy: 0.9476, Test Accuracy: 0.9476


In [ ]:
#from torchviz import make_dot
# x = torch.randn(1, 3, 224, 224)
# y = model(x)
# make_dot(y, params=dict(model.named_parameters())).render("model_architecture", format="png")

In [ ]:
def model_characteristics(model):
    num_params = sum(p.numel() for p in model.parameters())
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()
    buffer_size = 0
    for buffer in model.buffers():
        buffer_size += buffer.nelement() * buffer.element_size()
    size_all_mb = (param_size + buffer_size) / 1024**2
    print(f'Number of Parameters: {num_params:.3f}')
    print(f'Model size: {size_all_mb:.3f} MB')

Model size: 0.001 MB


In [160]:
save_dict = {
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'epoch': epoch,
    'loss': loss,
}

torch.save(save_dict, 'Fuzzy_model_checkpoint.pt')

In [ ]:
loaded_dict = torch.load('model_checkpoint.pt')

model.load_state_dict(loaded_dict['model_state_dict'])
optimizer.load_state_dict(loaded_dict['optimizer_state_dict'])
epoch = loaded_dict['epoch']
loss = loaded_dict['loss']